In [4]:
from azureml.core import Workspace, Experiment, Run, Model

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to work with', ws.name)

Ready to work with dsba6190ml


In [5]:
## Get experiment and run of interest
experiment = Experiment(workspace=ws, name='instructor_automl_diabetes')
run = Run(experiment, run_id = "AutoML_2f266fb1-d22a-41f4-9ec2-58ee52e1ae69_46")

In [6]:
run.get_metrics()

{'log_loss': 0.11075534524009302,
 'precision_score_weighted': 0.9564538972783089,
 'precision_score_micro': 0.9564570783554568,
 'f1_score_micro': 0.9564570783554568,
 'matthews_correlation': 0.9022974726242879,
 'recall_score_macro': 0.9507251765177537,
 'f1_score_weighted': 0.9564376425031265,
 'precision_score_binary': nan,
 'average_precision_score_micro': 0.9931689420359874,
 'balanced_accuracy': 0.9507251765177537,
 'recall_score_binary': nan,
 'AUC_macro': 0.9921501711655808,
 'f1_score_macro': 0.9511289284461748,
 'accuracy': 0.9564570783554568,
 'AUC_weighted': 0.9921501711655808,
 'AUC_binary': nan,
 'AUC_micro': 0.9929895504237377,
 'average_precision_score_macro': 0.9905396529134274,
 'weighted_accuracy': 0.9610813822458074,
 'precision_score_macro': 0.9515761766895965,
 'recall_score_weighted': 0.9564570783554568,
 'average_precision_score_weighted': 0.9923721633793233,
 'norm_macro_recall': 0.9014503530355075,
 'recall_score_micro': 0.9564570783554568,
 'average_precisio

In [8]:
## Register the model
print('Registering model...')
run.register_model(model_path='outputs/model.pkl', model_name='inst_automl_diabetes_model',
                   tags={'Training context':'Inline Training'},
                   properties={'AUC Weighed': run.get_metrics()['AUC_weighted'],
                               'Weighted Accuracy': run.get_metrics()['weighted_accuracy']})

## Get the registered model
model = ws.models['inst_automl_diabetes_model']

print('Model registered.')

Registering model...
Model registered.


## Deploy a model as a web service

In [9]:
import os

## Create a folder for the deployment files
deployment_folder = './diabetes_service'
os.makedirs(deployment_folder, exist_ok=True)
print(deployment_folder, 'folder created.')

## Set path for scoring script
script_file = 'score_diabetes.py'
script_path = os.path.join(deployment_folder,script_file)

./diabetes_service folder created.


Now you need an entry script that the service will use to score new data.

In [10]:
%%writefile $script_path
import json
import joblib
import numpy as np
import os

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'model.pkl')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = json.loads(raw_data)['data']
    np_data = np.array(data)
    # Get a prediction from the model
    predictions = model.predict(np_data)
    
    # print the data and predictions (so they'll be logged!)
    log_text = 'Data:' + str(data) + ' - Predictions:' + str(predictions)
    print(log_text)
    
    # Get the corresponding classname for each prediction (0 or 1)
    classnames = ['not-diabetic', 'diabetic']
    predicted_classes = []
    for prediction in predictions:
        predicted_classes.append(classnames[prediction])
    # Return the predictions as JSON
    return json.dumps(predicted_classes)

Writing ./diabetes_service/score_diabetes.py


Now you can deploy the service (in this case, as an Azure Container Instance (ACI).

> **Note**: This can take a few minutes - wait until the state is shown as **Healthy**.

In [22]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice

## Configure the scoring environment

curated_env_name = 'AzureML-pytorch-1.7-ubuntu18.04-py37-cuda11-gpu'
pytorch_env = Environment.get(workspace=ws, name=curated_env_name)

aci_service_env = Environment(name='inst_automl_aci_service-env')
python_packages = ['scikit-learn', 'azureml-defaults', 'azureml-sdk[automl]', 'azure-ml-api-sdk']
for package in python_packages:
    aci_service_env.python.conda_dependencies.add_pip_package(package)
inference_config = InferenceConfig(source_directory=deployment_folder,
                                   entry_script=script_file,
                                   environment=aci_service_env)

## Configure the web service container
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

In [ ]:
## Deploy the model as a service

print('Deploying model...')
service_name = "inst-automl-diabetes-service1"
aci_service = Model.deploy(ws, service_name, [model], inference_config, deployment_config, overwrite=True)
aci_service.wait_for_deployment(show_output = True)
print(aci_service.state)

Deploying model...


/tmp/ipykernel_26927/2140985576.py:5: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration. 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  aci_service = Model.deploy(ws, service_name, [model], inference_config, deployment_config, overwrite=True)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-03-18 16:31:52+00:00 Creating Container Registry if not exists.
2023-03-18 16:31:52+00:00 Registering the environment.
2023-03-18 16:31:53+00:00 Building image.

## Use the web service

With the service deployed, now you can consume it from a client application.

First, determine the URL to which these applications must submit their requests.

In [ ]:
endpoint = aci_service.scoring_uri
print(endpoint)

Now that you know the endpoint URI, an application can simply make an HTTP request, sending the patient data in JSON (or binary) format, and receive back the predicted class(es).

> **Tip**: If an error occurs because the service endpoint isn't ready. Wait a few seconds and try again!

In [ ]:
import requests
import json

# Create new data for inferencing
x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
         [0,148,58,11,179,39.19207553,0.160829008,45]]

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Set the content type
headers = { 'Content-Type':'application/json' }

# Get the predictions
predictions = requests.post(endpoint, input_json, headers = headers)
print(predictions.status_code)
if predictions.status_code == 200:
    predicted_classes = json.loads(predictions.json())
    for i in range(len(x_new)):
        print ("Patient {}".format(x_new[i]), predicted_classes[i] )